In [1]:
import matplotlib.pyplot as plt
import numpy as np
import osmnx as ox
import pandas as pd

ox.config(log_console=True, use_cache=True)
ox.__version__

'0.11.3'

In [2]:
# define the study sites as label : query
places = {'Boston'        : 'Boston, MA, USA',
          'Charlotte'     : 'Charlotte, NC, USA',
          'Chicago'       : 'Chicago, IL, USA',
          'Detroit'       : 'Detroit, MI, USA',
          'Manhattan'     : 'Manhattan, NYC, NY, USA',
          'Miami'         : 'Miami, FL, USA',
          'New Orleans'   : 'New Orleans, LA, USA',
          'Philadelphia'  : 'Philadelphia, PA, USA',
          'San Francisco' : {'city':'San Francisco', 'state':'CA', 'country':'USA'},
          'St Louis'      : 'St. Louis, MO, USA'}

In [3]:
# verify OSMnx geocodes each query to what you expect
gdf = ox.gdf_from_places(places.values())
gdf

,geometry,place_name,bbox_north,bbox_south,bbox_east,bbox_west
0,"POLYGON ((-71.19126 42.28284, -71.19107 42.282...","Boston, Suffolk County, Massachusetts, United ...",42.396977,42.227911,-70.804488,-71.191260
1,"POLYGON ((-81.00955 35.15144, -81.00953 35.151...","Charlotte, Mecklenburg County, North Carolina,...",35.393133,35.013174,-80.670104,-81.009554
2,"POLYGON ((-87.94010 42.00093, -87.94003 41.998...","Chicago, Cook County, Illinois, United States ...",42.023040,41.644531,-87.524081,-87.940101
3,"POLYGON ((-83.28796 42.44268, -83.28785 42.442...","Detroit, Wayne County, Michigan, United States...",42.450243,42.255192,-82.910439,-83.287959
4,"MULTIPOLYGON (((-74.04722 40.69086, -74.04722 ...","Manhattan, New York County, New York, United S...",40.880449,40.683941,-73.906159,-74.047222
5,"POLYGON ((-80.31976 25.76249, -80.31968 25.762...","Miami, Miami-Dade County, Florida, USA",25.855783,25.709052,-80.139157,-80.319760
6,"POLYGON ((-90.14003 29.94838, -90.13993 29.932...","New Orleans, Orleans Parish, Louisiana, USA",30.199469,29.865481,-89.625176,-90.140030
7,"POLYGON ((-75.28030 39.97500, -75.28022 39.974...","Philadelphia, Philadelphia County, Pennsylvani...",40.137959,39.867005,-74.955831,-75.280298
8,"MULTIPOLYGON (((-123.17382 37.77573, -123.1737...","San Francisco, San Francisco City and County, ...",37.929844,37.640314,-122.280016,-123.173825
9,"POLYGON ((-90.32065 38.59420, -90.31636 38.584...","St. Louis, City of Saint Louis, Missouri, Unit...",38.774340,38.532322,-90.166292,-90.320652


## Get the street networks and their edge bearings

In [4]:
def reverse_bearing(x):
    return x + 180 if x < 180 else x - 180

In [5]:
bearings = {}
for place in sorted(places.keys()):
    
    # get the graph
    query = places[place]
    G = ox.graph_from_place(query, network_type='drive')
    
    # calculate edge bearings on undirected graph
    Gu = ox.add_edge_bearings(ox.get_undirected(G))
    b = pd.Series([d['bearing'] for u, v, k, d in Gu.edges(keys=True, data=True)])
    bearings[place] = pd.concat([b, b.map(reverse_bearing)]).reset_index(drop='True')

## Visualize it

In [6]:
def count_and_merge(n, bearings):
    # make twice as many bins as desired, then merge them in pairs
    # prevents bin-edge effects around common values like 0° and 90°
    n = n * 2
    bins = np.arange(n + 1) * 360 / n
    count, _ = np.histogram(bearings, bins=bins)
    
    # move the last bin to the front, so eg 0.01° and 359.99° will be binned together
    count = np.roll(count, 1)
    return count[::2] + count[1::2]

In [7]:
# function to draw a polar histogram for a set of edge bearings
def polar_plot(ax, bearings, n=36, title=''):

    bins = np.arange(n + 1) * 360 / n
    count = count_and_merge(n, bearings)
    _, division = np.histogram(bearings, bins=bins)
    frequency = count / count.sum()
    division = division[0:-1]
    width =  2 * np.pi / n

    ax.set_theta_zero_location('N')
    ax.set_theta_direction('clockwise')

    x = division * np.pi / 180
    bars = ax.bar(x, height=frequency, width=width, align='center', bottom=0, zorder=2,
                  color='#003366', edgecolor='k', linewidth=0.5, alpha=0.7)
    
    ax.set_ylim(top=frequency.max())
    
    title_font = {'family':'Arial', 'size':24, 'weight':'bold'}
    xtick_font = {'family':'Arial', 'size':10, 'weight':'bold', 'alpha':1.0, 'zorder':3}
    ytick_font = {'family':'Arial', 'size': 9, 'weight':'bold', 'alpha':0.2, 'zorder':3}
    
    ax.set_title(title.upper(), y=1.05, fontdict=title_font)
    
    ax.set_yticks(np.linspace(0, max(ax.get_ylim()), 5))
    yticklabels = ['{:.2f}'.format(y) for y in ax.get_yticks()]
    yticklabels[0] = ''
    ax.set_yticklabels(labels=yticklabels, fontdict=ytick_font)
    
    xticklabels = ['N', '', 'E', '', 'S', '', 'W', '']
    ax.set_xticklabels(labels=xticklabels, fontdict=xtick_font)
    ax.tick_params(axis='x', which='major', pad=-2)

In [8]:
ncols = 5
nrows = 2
figsize = (ncols * 5, nrows * 5)
fig, axes = plt.subplots(nrows, ncols, figsize=figsize, subplot_kw={'projection':'polar'})

# plot each city's polar histogram
for ax, place in zip(axes.flat, sorted(places.keys())):
    polar_plot(ax, bearings[place].dropna(), title=place)

fig.tight_layout()
fig.subplots_adjust(hspace=0.35)
fig.savefig('images/street-orientations.png', dpi=120, bbox_inches='tight')
plt.close()

findfont: Font family ['Arial'] not found. Falling back to DejaVu Sans.
findfont: Font family ['Arial'] not found. Falling back to DejaVu Sans.
findfont: Font family ['Arial'] not found. Falling back to DejaVu Sans.
